In [6]:
import os
import glob
from tabulate import tabulate
from datetime import date
import pandas as pd
import numpy as np
import h5py as h5
from os import walk
from IPython.display import display, clear_output

class experiments:
    def __init__(self):
        self.headers=['No. Dir','Name Dir', 'No. files']

    def dataextraction(self, lab, sys, exptype, exp):
        self.expname=exp
        self.exptype=exptype
        self.lab=lab
        self.sys=sys
        self.path='/media/labfiles/lab-exps'
        self.foldername=[]
        self.filesname=[]
        self.pathname=[]
        self.ptable=[]
        self.datac=[]
        self.data=[]
        self.count=0
        self.error=0
        self.path=self.path+'/spectro-lab' if self.lab==1 else self.path+'/ellipsometry-lab'
        print(self.exptype)
        print(self.expname)
        if self.lab==1:
            self.path=self.path+'/nano' if self.sys=='nano' else self.path+'/cry-sys-2' if self.sys=='cry2' else self.path+'/cry-sys-1' if self.sys=='cry1' else self.path
    
        for (dirpath, dirnames, filenames) in walk(self.path):
            if self.expname in dirpath:
                self.foldername.append(dirpath)

        self.foldername = sorted(self.foldername)
        for dirpath in self.foldername:
            clear_output(wait=True)
            if (self.exptype in dirpath) and ('ccd' not in dirpath) and ('cpl' not in dirpath):
                self.datac=[]
                self.namef=[]
                for name in sorted(glob.glob(dirpath+'/*.xls')):
                    if self.exptype in name:
                        self.fu=name
                        dat=pd.read_excel(name).values
                        self.datac.append(dat[::-1])
                        self.namef.append(name)
                    else: pass
                for name in sorted(glob.glob(dirpath+'/*.h5')):
                    if self.exptype in name:
                        dat=h5.File(name,'r')
                        dset=list(dat.keys())
                        self.datac.append(dat[dset[0]][:,:])
                        self.namef.append(name)
                    else: pass
                for name in sorted(glob.glob(dirpath+'/*.dat')):
                    if (self.exptype in name and self.exptype=='pl'):
                        dat=np.loadtxt(name)
                        self.datac.append(dat[:,:])
                        self.namef.append(name)
                    else: pass
                self.data.append(self.datac)
                self.filesname.append(self.namef)
                self.pathname.append(dirpath)
                self.ptable.append([self.count,dirpath.split(self.path+'/')[1],len(self.datac)])
                self.count+=1
        dirpath
        if self.data:
            #print(tabulate(self.ptable,self.headers,tablefmt="github",colalign=("center","left","center")))
            pass
        else: 
            self.error=1
            #print("no experiments found, change parameters")        

    def h5creation(self):
        exptype=['ras','pr','pl','rd']
        samplespath="/media/labfiles/lab-exps/spectro-lab/cry-sys-1"
        samples=['m43521']
        uglyexceptions = ['m42696','m43598']
        for sample in samples:
            for type in exptype:
                self.help=[sample,type]
                if  sample in uglyexceptions:
                    pass
                else:
                    self.dataextraction(1,'cry1',type,sample)
                    if self.error: pass
                    else: 
                        months = []
                        days = []
                        for i in range(len(self.pathname)):
                            monthname = os.path.normpath(self.pathname[i]).split(os.path.sep)[7]
                            daysname = os.path.normpath(self.pathname[i]).split(os.path.sep)[8]
                            if (monthname not in months) and self.data[i]:
                                months.append(monthname)
                            else: pass
                            if (daysname not in days) and self.data[i]:
                                days.append(daysname)
                            else: pass

                        lend = len(self.data)
                        maxdimrows = 0
                        maxdimcols = 0
                        dimlist = []
                        #print(len(self.data))
                        for i in range(len(self.data)):
                            lend2 = len(self.data[i])
                            if lend2 > 0:
                                maxdimrows = max([len(j) for j in self.data[i]])
                                maxdimcols = max([self.data[i][k].shape[1] for k in range(len(self.data[i]))])
                                dimlist.append([maxdimrows,maxdimcols])
                            else: pass
                        
                        allarrs = []
                        for i in range(len(self.data)):
                            lend2 = len(self.data[i])
                            if lend2 > 0:
                                npa = np.empty((dimlist[i][0],dimlist[i][1],lend2), dtype=np.float64)
                                for j in range(lend2):
                                    rows,cols = self.data[i][j].shape
                                    npa[0:rows,0:cols,j] = self.data[i][j]
                                    
                                allarrs.append(npa)
                            else: pass

                        with h5.File(self.expname+".h5",'a') as file:
                            grps = []
                            dtsets = []
                            for i in range(len(months)):
                                group = file.require_group(months[i])
                                spltmonths = months[i].split('-',maxsplit=1)
                                for j in range(len(days)):
                                    if spltmonths[1] in days[j]: 
                                        datas = group.require_dataset(days[j],shape=allarrs[j].shape,data=allarrs[j],dtype=np.float64) 
                                        dtsets.append(datas)
                                grps.append(group)
                        
                        #print("File created succesfully")

    """def check4update():
        expspath = "/media/labfiles/lab-exps/spectro-lab/cry-sys-1"
        h5paths = "/media/labfiles/lab-exps/exp-h5-files/spectro-lab/cry-sys-1"
        updatesample = 0
        updategrps = 0
        updatefiles = 0
        exps = os.listdir(expspath)
        expsh5 = os.listdir(h5paths)
        if exps == expsh5:
            for i in range(len(exps)):
                with h5.File(expsh5[]) as f:
                gpsofldr = os.listdir(i)
        else: """

            



exp = experiments()
#test=exp.arraymaking


In [ ]:
exp.h5creation()#just call this function and creates h5 files for all samples